# Creacion del modelo con LSTM

Despues de analizar diversos metodos para realizar predicciones sobre este activo tan particular, se decidio utilizar redes neuronales recurrentes.

Long Short-Term Memory son un tipo especial de redes neuronales recurrentes (RNN) diseñadas para modelar secuencias de datos donde el orden de los elementos es crucial.

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import plotly_express as px
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import seaborn as sns
import datetime
from lightgbm import LGBMRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')